In [ ]:
# @title (Célula 1) Instalar o Pacote SVforensics
# @markdown Esta célula instala o pacote de software necessário diretamente do GitHub.
# @markdown **Para colaboradores externos:** Cole seu token de acesso fornecido oficialmente no campo abaixo.
# @markdown **Para uso interno (Polícia Científica):** Deixe o campo vazio.
# @markdown Pode levar um ou dois minutos para concluir.

# @markdown ---
# @markdown **Token de Acesso (apenas para colaboradores externos):**
TOKEN = "" # @param {type:"string"}

print("Instalando SVforensics...")

# Configurar URL de instalação baseada no token
if TOKEN.strip():
    # Para colaboradores externos com token
    install_url = f"git+https://{TOKEN}@github.com/sepai-dev/SVforensics.git"
    print("🔐 Usando autenticação via token para colaborador externo...")
else:
    # Para uso interno (conta organizacional)
    install_url = "git+https://github.com/sepai-dev/SVforensics.git"
    print("🏢 Usando acesso institucional interno...")

# -q para menos output, --no-cache-dir pode ajudar a evitar problemas de cache no Colab
%pip install --no-cache-dir {install_url} -q
print("✅ Instalação concluída!")

# Importar bibliotecas padrão que vamos precisar
import os
import json
import shutil
import logging
from pathlib import Path
from google.colab import files

# Configurar logging básico
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
logger = logging.getLogger("SVF_Colab")

print("Bibliotecas padrão importadas e logging configurado.")

In [ ]:
# @title (Célula 2) Downloads Essenciais e Leitura de Configuração
# @markdown Baixa arquivos de configuração, o modelo de embedding E os dados da população de referência.
# @markdown Lê as configurações para uso nas células seguintes.

import os
import json
import requests
from huggingface_hub import hf_hub_download
import logging
import time
from pathlib import Path

# Configurar logging básico
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
logger = logging.getLogger("SVF_Colab_Downloads")

# --- 1. Baixar Arquivos de Configuração ---
logger.info("Iniciando download dos arquivos de configuração...")

# Usar o mesmo token da célula anterior se disponível (para colaboradores externos)
try:
    auth_token = TOKEN.strip() if 'TOKEN' in globals() and TOKEN.strip() else None
except NameError:
    auth_token = None

# Configurar headers de autenticação se necessário
headers = {}
if auth_token:
    headers["Authorization"] = f"token {auth_token}"
    logger.info("🔐 Usando autenticação para download de arquivos de configuração")
else:
    logger.info("🏢 Usando acesso institucional para download de arquivos de configuração")

config_dir = "config"
os.makedirs(config_dir, exist_ok=True)
config_files_urls = {
    "svforensics.json": "https://raw.githubusercontent.com/sepai-dev/SVforensics/refs/heads/main/config/svforensics.json",
    "download_info.json": "https://raw.githubusercontent.com/sepai-dev/SVforensics/refs/heads/main/config/download_info.json",
    # <<< ADICIONADO plot_config.json >>>
    "plot_config.json": "https://raw.githubusercontent.com/sepai-dev/SVforensics/refs/heads/main/config/plot_config.json"
}
config_paths = {}
config_download_ok = True
for filename, url in config_files_urls.items():
    filepath = Path(config_dir) / filename
    try:
        logger.info(f"Baixando {filename} de {url}...")
        response = requests.get(url, headers=headers, timeout=20)
        response.raise_for_status()
        with open(filepath, 'wb') as f:
            f.write(response.content)
        config_paths[filename] = str(filepath)
        logger.info(f"- {filename} baixado com sucesso para {filepath}")
    except Exception as e:
        logger.error(f"Erro ao baixar {filename}: {e}")
        config_download_ok = False

if not config_download_ok:
    raise RuntimeError("Falha ao baixar arquivos de configuração essenciais.")
logger.info("Download dos arquivos de configuração concluído.")

# --- 2. Ler Configuração Principal ---
logger.info("Lendo configuração principal (svforensics.json)...")
config_main_path = config_paths.get("svforensics.json")
try:
    with open(config_main_path, 'r') as f:
        svf_config = json.load(f)
    logger.info("Configuração principal lida com sucesso.")
except Exception as e:
    raise RuntimeError(f"Falha ao ler o arquivo de configuração principal '{config_main_path}': {e}")

# --- 3. Baixar Modelo de Embedding ---
logger.info("Iniciando download do modelo de embedding...")
model_config = svf_config.get("model", {})
paths_config = svf_config.get("paths", {})
model_repo = model_config.get("repository", "Jenthe/ECAPA2")
model_filename = model_config.get("filename", "ecapa2.pt")
cache_dir = paths_config.get("models_cache_dir", "files/downloads/models")
os.makedirs(cache_dir, exist_ok=True)
try:
    logger.info(f"Baixando/verificando modelo '{model_filename}' de '{model_repo}' para '{cache_dir}'...")
    start_time = time.time()
    model_path = hf_hub_download(repo_id=model_repo, filename=model_filename, cache_dir=cache_dir)
    elapsed_time = time.time() - start_time
    logger.info(f"Modelo de embedding disponível em: {model_path} (Download/Verificação levou {elapsed_time:.2f}s)")
except Exception as e:
    raise RuntimeError(f"Falha ao baixar o modelo de embedding: {e}")

# --- 4. Baixar Dados da População de Referência ---
logger.info("Iniciando download dos dados da população de referência (definidos em download_info.json)...")
print("\n⏳ Baixando dados da população de referência (pode demorar)...")
try:
    from svforensics import download
    output_downloads_dir = paths_config.get("downloads_dir", "files/downloads")
    download_info_file_path = config_paths.get("download_info.json", "config/download_info.json") # Usa o baixado

    if not os.path.exists(download_info_file_path):
         logger.warning(f"Arquivo download_info.json não encontrado em {download_info_file_path}, pulando downloads adicionais.")
         print("⚠️ Arquivo de informações de download não encontrado. Dados de referência podem não ter sido baixados.")
    else:
        # Verifica se a função cli_main existe antes de chamar
        if hasattr(download, 'cli_main') and callable(getattr(download, 'cli_main')):
             download.cli_main(args=[
                 '--config', download_info_file_path,
                 '--output-dir', output_downloads_dir
             ])
             logger.info(f"Comando de download executado para '{output_downloads_dir}'.")
             print("✅ Download dos dados de referência concluído (ou verificado).")
        else:
             logger.error("Função download.cli_main não encontrada. Não foi possível executar downloads adicionais.")
             print("❌ Erro: Função de download não encontrada.")

except ImportError:
    logger.error("Erro ao importar o módulo 'download'.")
    print("❌ Erro: Não foi possível executar o download dos dados de referência.")
except Exception as e:
    logger.error(f"Erro durante o download dos dados de referência: {e}", exc_info=True)
    print("❌ Erro durante o download dos dados de referência.")

# --- 5. Salvar Configurações ---
logger.info("Salvando configurações lidas para uso posterior...")
%store svf_config paths_config config_paths model_repo model_filename cache_dir
logger.info("Configurações salvas.")

print("\nConfiguração inicial e downloads essenciais concluídos!")

In [ ]:
# @title (Célula 3) Mesclar Embeddings e Metadados da Referência
# @markdown Combina os embeddings e metadados da população de referência baixados.

import os
import logging

# Usar o logger configurado
logger = logging.getLogger("SVF_Colab_MergeRef")

# --- 1. Recuperar Variáveis Salvas ---
logger.info("Recuperando configurações salvas...")
try:
    %store -r svf_config
    %store -r paths_config
    logger.info("Variáveis recuperadas com sucesso.")
except KeyError as e:
    raise RuntimeError(f"Erro: Variável '{e}' não encontrada. Execute a Célula 2 primeiro.")

# --- 2. Importar Funcionalidade de Merge ---
logger.info("Importando a funcionalidade de merge...")
try:
    from svforensics import metadata_embedding_merge
    logger.info("Módulo 'metadata_embedding_merge' importado.")
except ImportError:
    raise RuntimeError("Erro ao importar 'metadata_embedding_merge'. Verifique a instalação.")

# --- 3. Definir Caminhos de Entrada e Saída ---
logger.info("Definindo caminhos para o merge...")
# Pegar caminhos dos arquivos baixados na Célula 2 da config
ref_embedding_file = paths_config.get("raw_embeddings_file", "files/downloads/vox1_test_whatsapp_ecapa2.pth")
ref_metadata_file = paths_config.get("metadata_file", "files/downloads/vox1_meta.csv")
# Definir prefixo de saída para o arquivo mesclado (.pth será adicionado)
output_prefix = paths_config.get("output_prefix", "files/generated/metadata_embeddings/processed_embeddings")
# Colunas a remover (da config)
drop_columns = svf_config.get("processing", {}).get("drop_columns", None)

# Verificar se os arquivos de entrada existem
if not os.path.exists(ref_embedding_file) or not os.path.exists(ref_metadata_file):
    logger.error(f"Arquivos de entrada para merge não encontrados: Emb='{ref_embedding_file}', Meta='{ref_metadata_file}'")
    raise FileNotFoundError("Arquivos de referência baixados na Célula 2 não encontrados.")

logger.info(f"Arquivo de embeddings de referência: {ref_embedding_file}")
logger.info(f"Arquivo de metadados de referência: {ref_metadata_file}")
logger.info(f"Prefixo de saída para dados mesclados: {output_prefix}")

# Garantir que o diretório de saída exista
os.makedirs(os.path.dirname(output_prefix), exist_ok=True)

# --- 4. Executar Merge ---
logger.info("Iniciando o merge dos dados de referência...")
print("\n⏳ Mesclando dados da população de referência...")
try:
    # Usar a função main do módulo, que encapsula a lógica
    merged_df = metadata_embedding_merge.main(
        embedding_file=ref_embedding_file,
        metadata_file=ref_metadata_file,
        output_prefix=output_prefix,
        test_file=None, # Não estamos filtrando por test list aqui
        drop_columns=drop_columns, # Usa as colunas da config
        save_output=True # Queremos salvar o .pth mesclado
    )
    output_merged_file = f"{output_prefix}.pth" # Caminho do arquivo salvo
    logger.info(f"Merge concluído. Dados mesclados salvos em {output_merged_file}")
    print(f"✅ Merge dos dados de referência concluído. Salvo em: {output_merged_file}")

    # Salvar caminho do arquivo mesclado para a próxima célula
    %store output_merged_file

except Exception as e:
    logger.error(f"Erro durante o merge: {e}", exc_info=True)
    print("❌ Erro durante o merge dos dados de referência. Verifique os logs.")
    raise e


In [ ]:
# @title (Célula 4) Gerar Lista de Teste da População de Referência
# @markdown Cria pares de comparação (mesmo/diferente locutor) da população de referência.
# @markdown **Requer que você selecione o gênero abaixo.**

import os
import logging
from pathlib import Path # Importar Path

# @markdown ---
# @markdown ### Selecione o Gênero para a Lista de Teste:
# @markdown A lista de teste será criada usando apenas locutores do gênero selecionado.
selected_gender = "m" # @param ["m", "f"] {allow-input: false}
# @markdown ---

# Usar o logger configurado
logger = logging.getLogger("SVF_Colab_Testlist")

# --- 1. Recuperar Variáveis Salvas ---
logger.info("Recuperando configurações e caminho dos dados mesclados...")
try:
    %store -r svf_config
    %store -r paths_config
    # Correção: Remover comentário desta linha
    %store -r output_merged_file
    logger.info("Variáveis recuperadas com sucesso.")
except KeyError as e:
    raise RuntimeError(f"Erro: Variável '{e}' não encontrada. Execute as Células 2 e 3 primeiro.")

# --- 2. Importar Funcionalidade de Testlist ---
logger.info("Importando a funcionalidade de geração de testlist...")
try:
    from svforensics import testlists
    logger.info("Módulo 'testlists' importado.")
except ImportError:
    raise RuntimeError("Erro ao importar 'testlists'. Verifique a instalação.")

# --- 3. Definir Parâmetros ---
logger.info("Definindo parâmetros para geração da testlist...")
merged_embeddings_file = output_merged_file
output_testlist_prefix = paths_config.get("test_list_prefix", "files/generated/testlists/test_list")
testlist_config = svf_config.get("testlists", {})
n_pos = testlist_config.get("n_pos", 1)
n_neg = testlist_config.get("n_neg", 1)
different_videos = testlist_config.get("different_videos", True)
test_prop = testlist_config.get("test_prop", 0.5)
random_state = testlist_config.get("random_seed", 42)
gender = selected_gender

# Log dos parâmetros (opcional, pode remover para menos verbosidade)
# logger.info(f"Gênero selecionado para a lista: {gender}")
# logger.info(f"Arquivo de embeddings mesclados (entrada): {merged_embeddings_file}")
# logger.info(f"Prefixo de saída para a lista de teste: {output_testlist_prefix}")
# logger.info(f"n_pos={n_pos}, n_neg={n_neg}, different_videos={different_videos}, test_prop={test_prop}, seed={random_state}")

if not os.path.exists(merged_embeddings_file):
    raise FileNotFoundError(f"Arquivo de embeddings mesclados não encontrado: {merged_embeddings_file}")

os.makedirs(Path(output_testlist_prefix).parent, exist_ok=True) # Usar Path

# --- 4. Executar Geração da Testlist ---
# Manter o print inicial para o usuário saber o que está acontecendo
print(f"\n⏳ Gerando lista de teste para gênero '{gender}'...")
try:
    generated_testlist_path = testlists.create_test_lists(
        gender=gender,
        embeddings_file=merged_embeddings_file,
        output_prefix=output_testlist_prefix,
        n_pos=n_pos,
        n_neg=n_neg,
        different_videos=different_videos,
        test_prop=test_prop,
        random_state=random_state
    )
    logger.info(f"Geração da lista de teste concluída. Salva em {generated_testlist_path}")
    # Manter o print de sucesso para o usuário
    print(f"✅ Lista de teste gerada com sucesso! Salva em: {generated_testlist_path}")

    %store generated_testlist_path

except Exception as e:
    logger.error(f"Erro durante a geração da lista de teste: {e}", exc_info=True)
    # Manter o print de erro para o usuário
    print("❌ Erro durante a geração da lista de teste. Verifique os logs.")
    raise e


In [ ]:
# @title (Célula 5) Upload dos Arquivos de Áudio do Caso
# @markdown Faça o upload dos arquivos 'questionado(s)' e 'referência(s)' **deste caso específico**.
# @markdown Os arquivos serão movidos para os diretórios corretos.
# @markdown ---
# @markdown 1. **Questionados:** Clique em "Escolher arquivos" e selecione **todos** os arquivos questionados.
# @markdown 2. **Referência:** Clique em "Escolher arquivos" novamente e selecione **todos** os arquivos de referência.

import os
import shutil
from pathlib import Path
from google.colab import files
import logging
import glob

# Usar o logger configurado
logger = logging.getLogger("SVF_Colab_Upload_Caso")

# --- Nomes Definidos para os Diretórios do Caso ---
dir_caso_q = "audios_caso_questionados"
dir_caso_r = "audios_caso_referencia"
logger.info(f"Diretórios de upload do caso definidos: '{dir_caso_q}', '{dir_caso_r}'")

# --- Limpeza Prévia ---
for d in [dir_caso_q, dir_caso_r]:
    if os.path.exists(d):
        logger.warning(f"Limpando diretório de caso existente: {d}")
        shutil.rmtree(d)
    os.makedirs(d, exist_ok=True)
    logger.info(f"Diretório de caso criado/limpo: {d}")

def upload_e_renomear(diretorio_destino, tipo_arquivo_log):
    """Função auxiliar para solicitar upload e MOVER usando os.rename."""
    print(f"\n➡️ Por favor, faça o upload do(s) arquivo(s) de áudio {tipo_arquivo_log.upper()} (DO CASO):")
    uploaded_filenames = []
    try:
        uploaded = files.upload()
        if not uploaded:
            logger.warning(f"Nenhum arquivo {tipo_arquivo_log} do caso foi enviado.")
            print(f"⚠️ Nenhum arquivo {tipo_arquivo_log} selecionado.")
            return False, 0, []
        else:
            count_moved = 0
            uploaded_filenames = list(uploaded.keys())
            for filename in uploaded_filenames:
                source_path_str = filename
                dest_path_str = str(Path(diretorio_destino) / filename)
                try:
                    os.rename(source_path_str, dest_path_str)
                    logger.info(f"Arquivo {tipo_arquivo_log} '{filename}' movido para '{dest_path_str}'.")
                    count_moved += 1
                except OSError as e:
                    logger.error(f"Erro ao MOVER (os.rename) o arquivo {filename} para {dest_path_str}: {e}")
                    print(f"❌ Erro ao mover {filename}.")
                    if os.path.exists(source_path_str):
                         logger.warning(f"Arquivo {filename} ainda existe na raiz após falha no os.rename.")

            if count_moved == len(uploaded):
                print(f"✅ {count_moved} arquivo(s) {tipo_arquivo_log} do caso processado(s) com sucesso.")
                return True, count_moved, uploaded_filenames
            else:
                 print(f"⚠️ {count_moved}/{len(uploaded)} arquivo(s) {tipo_arquivo_log} do caso processado(s). Verifique os logs.")
                 return False, count_moved, uploaded_filenames

    except Exception as e:
        logger.error(f"Erro durante o processo de upload/movimentação para {tipo_arquivo_log}: {e}")
        print(f"❌ Ocorreu um erro inesperado durante o upload.")
        return False, 0, uploaded_filenames

# --- Executar Upload ---
upload_ok_q, count_q, files_q = upload_e_renomear(dir_caso_q, "questionado(s)")
upload_ok_r, count_r, files_r = False, 0, []
if upload_ok_q and count_q > 0:
    upload_ok_r, count_r, files_r = upload_e_renomear(dir_caso_r, "referência")
elif not upload_ok_q:
     print("\nUpload de arquivos questionados do caso falhou ou foi cancelado.")
else:
    print("\nNenhum arquivo questionado do caso foi selecionado.")

# --- Verificação Final e Armazenamento ---
upload_geral_ok = upload_ok_q and count_q > 0 and upload_ok_r and count_r > 0

if upload_geral_ok:
    logger.info(f"Upload e movimentação do caso concluídos: {count_q} questionado(s), {count_r} referência(s).")
    # Correção: Remover o -q inválido
    %store dir_caso_q
    %store dir_caso_r
    print("\n🎉 Uploads do caso concluídos com sucesso!")
else:
    logger.error("Upload do caso falhou ou arquivos essenciais não foram enviados/movidos.")
    print("\n❌ Atenção: Upload do caso não concluído corretamente.")

# --- LIMPEZA FINAL DA RAIZ ---
logger.info("Limpando arquivos temporários da pasta raiz...")
files_to_clean = files_q + files_r
cleaned_count = 0
errors_cleaning = 0

for filename in files_to_clean:
     if os.path.exists(filename):
          try:
               os.remove(filename)
               logger.info(f"Arquivo remanescente '{filename}' removido da raiz.")
               cleaned_count += 1
          except OSError as e:
               logger.error(f"Erro ao remover arquivo remanescente '{filename}' da raiz: {e}")
               errors_cleaning += 1

extra_ogg_files = glob.glob("*.ogg")
for extra_file in extra_ogg_files:
     if os.path.exists(extra_file):
          logger.warning(f"Encontrado arquivo .ogg inesperado '{extra_file}' na raiz. Tentando remover...")
          try:
               os.remove(extra_file)
               logger.info(f"Arquivo extra '{extra_file}' removido da raiz.")
               cleaned_count += 1
          except OSError as e:
               logger.error(f"Erro ao remover arquivo extra '{extra_file}' da raiz: {e}")
               errors_cleaning += 1

logger.info(f"Limpeza da raiz finalizada. Removidos: {cleaned_count}, Erros: {errors_cleaning}")

if not upload_geral_ok:
     print("\nLembrete: O upload não foi totalmente bem-sucedido.")


In [ ]:
# @title (Célula 6) Preparar Arquivos de Áudio do Caso
# @markdown Processa os áudios originais do caso, salvando chunks diretamente em 'probe' e 'reference'.

import os
from pathlib import Path
import logging
import shutil
import glob # Para encontrar arquivos

# Usar o logger configurado
logger = logging.getLogger("SVF_Colab_AudioPrep_Caso")

# --- 1. Recuperar Variáveis Salvas ---
logger.info("Recuperando variáveis salvas...")
try:
    # Correção: Remover comentários das linhas %store -r
    %store -r dir_caso_q
    %store -r dir_caso_r
    %store -r svf_config
    %store -r paths_config
    logger.info("Variáveis recuperadas com sucesso.")
except KeyError as e:
    raise RuntimeError(f"Erro: Variável '{e}' não encontrada. Execute as Células 2 e 5 primeiro.")

# --- 2. Importar Funções de Preparação ---
logger.info("Importando funções de preparação de áudio...")
try:
    from svforensics.audioprep import process_audio_file, DEFAULT_AUDIO_EXTENSIONS, DEFAULT_SAMPLE_RATE, DEFAULT_CHUNK_DURATION, DEFAULT_FADE_DURATION
    logger.info("Função 'process_audio_file' e constantes importadas.")
except ImportError as e:
    raise RuntimeError(f"Erro ao importar de svforensics.audioprep: {e}.")

# --- 3. Definir Parâmetros e Caminhos de Saída ---
logger.info("Definindo parâmetros e caminhos de saída...")

audio_config = svf_config.get("audio", {})
sample_rate = audio_config.get("sample_rate", DEFAULT_SAMPLE_RATE)
chunk_duration = audio_config.get("chunk_duration", DEFAULT_CHUNK_DURATION)
fade_duration = audio_config.get("fade_duration", DEFAULT_FADE_DURATION)
min_chunk_duration = audio_config.get("min_chunk_duration")
output_format = audio_config.get("output_format", "wav")
audio_extensions = audio_config.get("audio_extensions", DEFAULT_AUDIO_EXTENSIONS)

output_probe_chunk_dir = Path(paths_config.get("probe_processed_dir", "files/generated/case/audio_chunks/probe"))
output_ref_chunk_dir = Path(paths_config.get("reference_processed_dir", "files/generated/case/audio_chunks/reference"))
logger.info(f"Diretório de saída para chunks Questionados: {output_probe_chunk_dir}")
logger.info(f"Diretório de saída para chunks Referência: {output_ref_chunk_dir}")

# --- 4. Limpar Diretórios de Saída e Processar Arquivos ---
all_processed_chunks = []
total_probe_chunks = 0
total_ref_chunks = 0

# Manter o print para feedback
print("\n⏳ Iniciando preparação dos áudios do caso (processamento direto)...")

for input_dir_name, output_chunk_dir, type_label in [
    (dir_caso_q, output_probe_chunk_dir, "Questionado"),
    (dir_caso_r, output_ref_chunk_dir, "Referência")
]:
    logger.info(f"Processando tipo: {type_label}")
    if output_chunk_dir.exists():
        logger.warning(f"Limpando diretório de saída existente: {output_chunk_dir}")
        shutil.rmtree(output_chunk_dir)
    output_chunk_dir.mkdir(parents=True, exist_ok=True)
    logger.info(f"Diretório de saída '{output_chunk_dir}' limpo/criado.")

    input_files = []
    for ext in audio_extensions:
        pattern = str(Path(input_dir_name) / f"*{ext}")
        input_files.extend(glob.glob(pattern))

    if not input_files:
        logger.warning(f"Nenhum arquivo de áudio encontrado em '{input_dir_name}' com extensões {audio_extensions}")
        continue
    logger.info(f"Encontrados {len(input_files)} arquivos em '{input_dir_name}'. Processando...")

    processed_count_type = 0
    for audio_file in input_files:
        try:
            chunk_files = process_audio_file(
                audio_file=audio_file,
                output_dir=str(output_chunk_dir),
                sample_rate=sample_rate,
                chunk_duration=chunk_duration,
                fade_duration=fade_duration,
                min_chunk_duration=min_chunk_duration,
                format=output_format
            )
            all_processed_chunks.extend(chunk_files)
            processed_count_type += len(chunk_files)
        except Exception as e:
            logger.error(f"Falha ao processar arquivo {audio_file}: {e}", exc_info=True)
            # Manter print de erro para o usuário
            print(f"❌ Erro ao processar {os.path.basename(audio_file)}. Verifique os logs.")

    logger.info(f"Processamento do tipo {type_label} concluído. {processed_count_type} chunks criados.")
    if type_label == "Questionado":
        total_probe_chunks = processed_count_type
    else:
        total_ref_chunks = processed_count_type

# Manter prints de feedback
print(f"\n✅ Preparação do caso concluída!")
print(f"   - {total_probe_chunks} segmentos de fala extraídos dos áudios questionados.")
print(f"   - {total_ref_chunks} segmentos de fala extraídos dos áudios de referência.")
print(f"   - Arquivos processados (.wav) salvos em: '{output_probe_chunk_dir}' e '{output_ref_chunk_dir}'")

# --- 5. Salvar Caminhos para a Próxima Célula ---
parent_chunk_dir_probe_caso = str(output_probe_chunk_dir)
parent_chunk_dir_ref_caso = str(output_ref_chunk_dir)

logger.info(f"Salvando caminhos dos diretórios de chunks do caso...")
%store parent_chunk_dir_probe_caso
%store parent_chunk_dir_ref_caso
logger.info("Caminhos salvos.")

# Manter verificação no final (opcional)
# print("\nVerificando diretórios de saída:")
# print(f"Conteúdo de '{parent_chunk_dir_probe_caso}':")
# !ls -l "{parent_chunk_dir_probe_caso}"
# print(f"\nConteúdo de '{parent_chunk_dir_ref_caso}':")
# !ls -l "{parent_chunk_dir_ref_caso}"


In [ ]:
# @title (Célula 7) Extrair Embeddings do Caso
# @markdown Analisa os segmentos de áudio (.wav) do caso e extrai as características (embeddings).
# @markdown **Este processo pode ser demorado**.

import os
import logging
import time
from pathlib import Path
from tqdm import tqdm
import torch

# Usar o logger configurado
logger = logging.getLogger("SVF_Colab_Embedding_Caso")

# --- 1. Recuperar Variáveis Salvas ---
logger.info("Recuperando variáveis salvas das células anteriores...")
try:
    # Diretórios PAI contendo subdirs com chunks .wav DO CASO (da Célula 6)
    %store -r parent_chunk_dir_probe_caso
    %store -r parent_chunk_dir_ref_caso
    # Nomes dos diretórios originais DO CASO (da Célula 5, usados como 'speaker_id')
    %store -r dir_caso_q
    %store -r dir_caso_r
    # Configurações e parâmetros do modelo
    %store -r svf_config
    %store -r paths_config
    %store -r model_repo
    %store -r model_filename
    %store -r cache_dir
    logger.info("Variáveis recuperadas com sucesso.")
except KeyError as e:
    raise RuntimeError(f"Erro: Variável '{e}' não encontrada. Execute as células anteriores primeiro.")

# --- 2. Importar Classe e Instanciar Extrator ---
logger.info("Importando e instanciando EmbeddingExtractor...")
try:
    from svforensics.case_embeddings import EmbeddingExtractor
    start_time = time.time()
    embedding_extractor = EmbeddingExtractor(
        model_repo=model_repo,
        model_filename=model_filename,
        cache_dir=cache_dir,
        use_half_precision=False
    )
    _ = embedding_extractor.model # Carrega modelo do cache
    elapsed_time = time.time() - start_time
    logger.info(f"EmbeddingExtractor instanciado e modelo carregado de '{cache_dir}' em {elapsed_time:.2f}s.")
    logger.info(f"Usando dispositivo: {embedding_extractor.device}")
except ImportError:
     raise RuntimeError("Erro ao importar EmbeddingExtractor.")
except Exception as e:
    raise RuntimeError(f"Erro ao instanciar EmbeddingExtractor ou carregar modelo: {e}")

# --- 3. Encontrar Arquivos de Chunk do Caso ---
logger.info("Procurando por arquivos de chunk (.wav) do caso...")
# Usa os diretórios pai _caso salvos na Célula 6
probe_chunk_files = list(Path(parent_chunk_dir_probe_caso).rglob('*.wav'))
ref_chunk_files = list(Path(parent_chunk_dir_ref_caso).rglob('*.wav'))
all_chunk_files = probe_chunk_files + ref_chunk_files
total_chunks = len(all_chunk_files)
logger.info(f"Encontrados {len(probe_chunk_files)} chunks questionados e {len(ref_chunk_files)} chunks de referência do caso.")
logger.info(f"Total de {total_chunks} chunks do caso a processar.")

if total_chunks == 0:
    raise RuntimeError("Nenhum arquivo de chunk (.wav) do caso encontrado. A Célula 6 falhou?")

# --- 4. Preparar Dicionários de Saída ---
# Usa os nomes dos diretórios originais do CASO como 'speaker_id'
probe_speaker_id = Path(dir_caso_q).name
ref_speaker_id = Path(dir_caso_r).name
probe_embeddings_output = {probe_speaker_id: {}}
reference_embeddings_output = {ref_speaker_id: {}}
logger.info(f"Estrutura de saída preparada para speakers do caso: '{probe_speaker_id}', '{ref_speaker_id}'")

# --- 5. Executar Extração (Loop Único com Progresso) ---
logger.info("Iniciando extração de embeddings do caso (loop único)...")
print(f"\n⏳ Iniciando extração de embeddings para {total_chunks} arquivos do caso (pode levar vários minutos)...")

extraction_errors = 0
start_time_extraction = time.time()
for file_path_obj in tqdm(all_chunk_files, desc="Extraindo Embeddings (Caso)", unit="file"):
    file_path_str = str(file_path_obj)
    try:
        embedding = embedding_extractor.extract_embedding(file_path_str)
        # Usa os diretórios pai _caso para determinar onde colocar
        if file_path_str.startswith(parent_chunk_dir_probe_caso):
            probe_embeddings_output[probe_speaker_id][file_path_str] = embedding.cpu()
        elif file_path_str.startswith(parent_chunk_dir_ref_caso):
            reference_embeddings_output[ref_speaker_id][file_path_str] = embedding.cpu()
        else:
            logger.warning(f"Arquivo {file_path_str} não pertence a probe nem reference do caso? Ignorando.")
    except Exception as e:
        logger.error(f"Falha ao extrair embedding de {file_path_str}: {e}")
        extraction_errors += 1

elapsed_time_extraction = time.time() - start_time_extraction
logger.info(f"Extração do caso concluída em {elapsed_time_extraction:.2f}s com {extraction_errors} erro(s).")

if extraction_errors > 0:
     print(f"\n⚠️ Atenção: {extraction_errors} erro(s) ocorreram durante a extração do caso.")

total_probe_extracted = len(probe_embeddings_output[probe_speaker_id])
total_ref_extracted = len(reference_embeddings_output[ref_speaker_id])
print("\n✅ Extração de embeddings do caso concluída!")
print(f"   - {total_probe_extracted} embeddings extraídos dos áudios questionados do caso.")
print(f"   - {total_ref_extracted} embeddings extraídos dos áudios de referência do caso.")

# --- 6. Salvar Resultados ---
# Pegar os caminhos de saída definidos na config para os embeddings do CASO
output_embeddings_dir = paths_config.get("embeddings_dir", "files/generated/case/embeddings")
os.makedirs(output_embeddings_dir, exist_ok=True)
# Estes são os nomes de arquivo padrão para os embeddings do CASO
output_probe_emb_file_caso = paths_config.get("probe_embeddings_file", os.path.join(output_embeddings_dir,"probe_embeddings.pt"))
output_ref_emb_file_caso = paths_config.get("reference_embeddings_file", os.path.join(output_embeddings_dir,"reference_embeddings.pt"))

logger.info(f"Salvando embeddings do caso em {output_embeddings_dir}...")
try:
    torch.save(probe_embeddings_output, output_probe_emb_file_caso)
    logger.info(f"Embeddings Questionados do caso salvos em: {output_probe_emb_file_caso}")
    torch.save(reference_embeddings_output, output_ref_emb_file_caso)
    logger.info(f"Embeddings Referência do caso salvos em: {output_ref_emb_file_caso}")
    print(f"   - Arquivos de embeddings do caso salvos em: {output_embeddings_dir}")

    # Salvar caminhos para a célula de análise final (Célula 8)
    # Usa os nomes distintos com _caso
    %store output_probe_emb_file_caso
    %store output_ref_emb_file_caso
    logger.info("Caminhos dos arquivos de embeddings do caso salvos.")

except Exception as e:
    logger.error(f"Erro ao salvar arquivos de embeddings .pt do caso: {e}", exc_info=True)
    print(f"\n❌ Erro ao salvar arquivos de embeddings do caso.")
    raise e

In [ ]:
# @title (Célula 8) Análise Final e Plotagem
# @markdown Compara os embeddings do caso com os de referência populacional e visualiza os resultados.

import os
import logging
import matplotlib.pyplot as plt
import time
from pathlib import Path
from svforensics import config

# Usar o logger configurado
logger = logging.getLogger("SVF_Colab_Analysis_Final")

# --- 1. Recuperar Variáveis Salvas ---
logger.info("Recuperando variáveis salvas...")
try:
    %store -r output_probe_emb_file_caso
    %store -r output_ref_emb_file_caso
    %store -r generated_testlist_path
    %store -r output_merged_file
    %store -r svf_config
    %store -r paths_config
    logger.info("Variáveis recuperadas com sucesso.")
except KeyError as e:
     raise RuntimeError(f"Erro: Variável essencial '{e}' não encontrada. Execute as células anteriores.")

# --- 2. Importar Funções Necessárias ---
logger.info("Importando funções de verificação/análise...")
try:
    from svforensics.verification import (
        load_test_embeddings, calculate_test_scores, analyze_scores_distribution,
        compare_case_embeddings, plot_results, interpret_results # interpret_results ainda é importado, mas não usado para print
    )
    logger.info("Funções importadas com sucesso.")
except ImportError as e:
    raise RuntimeError(f"Erro ao importar de svforensics.verification: {e}. Verifique a instalação.")

# --- 3. Definir Parâmetros e Validar Entradas ---
logger.info("Definindo parâmetros e verificando arquivos de entrada...")
test_list_path = generated_testlist_path
processed_embeddings_file = output_merged_file
probe_embeddings_file = output_probe_emb_file_caso
reference_embeddings_file = output_ref_emb_file_caso

use_population_data = False
if test_list_path and os.path.exists(test_list_path):
    if processed_embeddings_file and os.path.exists(processed_embeddings_file):
        logger.info(f"Usando dados populacionais: Lista={test_list_path}, Embeddings={processed_embeddings_file}")
        use_population_data = True
    else:
        logger.warning(f"Lista de teste encontrada ({test_list_path}), mas arquivo de embeddings processados da população não ({processed_embeddings_file}). Análise populacional desativada.")
else:
    logger.warning(f"Lista de teste populacional não encontrada ou não definida ({test_list_path}). Análise populacional desativada.")

use_case_data = False
if probe_embeddings_file and reference_embeddings_file and \
   os.path.exists(probe_embeddings_file) and os.path.exists(reference_embeddings_file):
    logger.info(f"Usando dados do caso: Probe={probe_embeddings_file}, Ref={reference_embeddings_file}")
    use_case_data = True
else:
    logger.warning(f"Arquivos de embeddings do caso não encontrados (Probe: {probe_embeddings_file}, Ref: {reference_embeddings_file}). Análise do caso desativada.")

if not use_population_data and not use_case_data:
    raise RuntimeError("Nenhum dado válido encontrado para análise.")

output_plot_file = paths_config.get("case_analysis_plot", "files/plots/case_analysis.png")
os.makedirs(Path(output_plot_file).parent, exist_ok=True)
plot_config_path = config_paths.get("plot_config.json", config.DEFAULT_PLOT_CONFIG_PATH) # Usar o caminho baixado
logger.info(f"Usando configuração de plot: {plot_config_path}")
language = config.get_default_language(config=svf_config)
show_progress = True
logger.info(f"Idioma para plotagem: {language}")

# --- 4. Executar Análise Populacional (se aplicável) ---
same_speaker_stats = {"count": 0, "scores": [], "mean": None, "std": None, "min": None, "max": None, "median": None}
diff_speaker_stats = {"count": 0, "scores": [], "mean": None, "std": None, "min": None, "max": None, "median": None}

if use_population_data:
    logger.info("Iniciando análise populacional...")
    # Manter print de status
    print("\n⏳ Calculando scores da população de referência...")
    try:
        start_time = time.time()
        pop_embeddings, test_pairs = load_test_embeddings(test_list_path, processed_embeddings_file)
        if pop_embeddings:
            pop_results = calculate_test_scores(pop_embeddings, test_pairs, show_progress)
            same_speaker_stats, diff_speaker_stats = analyze_scores_distribution(pop_results)
            elapsed_time = time.time() - start_time
            logger.info(f"Análise populacional concluída em {elapsed_time:.2f}s.")
            print("✅ Análise populacional concluída.") # Manter
        else:
            logger.warning("Não foi possível carregar embeddings da população.")
            print("⚠️ Embeddings da população não carregados.") # Manter
    except Exception as e:
        logger.error(f"Erro na análise populacional: {e}", exc_info=True)
        print("❌ Erro durante a análise populacional.") # Manter

# --- 5. Executar Análise do Caso (se aplicável) ---
case_results = None
if use_case_data:
    logger.info("Iniciando análise do caso...")
    # Manter print de status
    print("\n⏳ Comparando embeddings do caso...")
    try:
        start_time = time.time()
        case_results = compare_case_embeddings(
            probe_embeddings_file, reference_embeddings_file, show_progress=show_progress
        )
        elapsed_time = time.time() - start_time
        logger.info(f"Análise do caso concluída em {elapsed_time:.2f}s.")
        print("✅ Comparação do caso concluída.") # Manter
    except Exception as e:
        logger.error(f"Erro na análise do caso: {e}", exc_info=True)
        print("❌ Erro durante a comparação do caso.") # Manter

# --- 6. Gerar Plot ---
logger.info("Gerando o gráfico...")
# Manter print de status
print("\n📊 Gerando gráfico...")
plot_generated = False
try:
    plot_results(
        same_speaker_stats=same_speaker_stats,
        diff_speaker_stats=diff_speaker_stats,
        case_results=case_results,
        output_file=output_plot_file,
        config_path=plot_config_path,
        language=language
    )
    if os.path.exists(output_plot_file):
        plot_generated = True
        logger.info(f"Gráfico salvo em {output_plot_file}")
        print(f"✅ Gráfico salvo em {output_plot_file}") # Manter
    else:
         logger.warning("Função plot_results executada, mas arquivo de plot não encontrado.")
         print("⚠️ Arquivo de gráfico não foi criado.") # Manter

except Exception as e:
    logger.error(f"Erro ao gerar o gráfico: {e}", exc_info=True)
    print("❌ Erro durante a geração do gráfico.") # Manter

# --- 7. Exibir Plot e Interpretação ---
if plot_generated:
    logger.info("Exibindo o gráfico gerado...")
    from IPython.display import Image, display
    display(Image(filename=output_plot_file))
else:
    # Manter aviso se não plotou
    print("\n⚠️ O gráfico não pôde ser exibido.")

# <<<<< REMOVIDA A SEÇÃO DE INTERPRETAÇÃO >>>>>
# print("\n--- Interpretação dos Resultados ---")
# final_results_for_interpretation = {
#     "same_speaker": same_speaker_stats,
#     "different_speaker": diff_speaker_stats,
#     "case_results": case_results,
#     "plot_file": output_plot_file if plot_generated else None
# }
# interpretation_text = interpret_results(final_results_for_interpretation)
# print(interpretation_text)
# print("------------------------------------")

# Manter mensagem final de sucesso
print("\n🎉 Análise concluída!")
